<a href="https://colab.research.google.com/github/yi-ye-zhi-qiu/kitchensoundscapes/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Autoencoder



1.   Get note/chords data from midi folder of files
*Reproducing?* Create a "music" folder in your Drive, and "training_songs" folder in that. Store training data there
2.   Convert training data into a format Autoencoder can use (one hot encode notes/chords, etc.)
3.   Train Autoencoder
4.   Let Autoencoder "reconstruct" the "image" of data that we feed it. If you feed it a song called "tomato soup", it will reconstruct tomato soup. So on and so forth.
5.   Take that reconstruction, and save as a generated MIDI file named as today's date/time.



In [2]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Flatten, Reshape
from keras import layers
from keras.models import Model
from keras.optimizers import RMSprop

import tensorflow as tf

from music21 import converter, instrument, note, chord, stream
import pickle
import os
import numpy as np

Get notes from MIDI files
- Note: music21 differentiates *chords* with *notes*, **chords** (the occurrence of multiple notes @ once) are interpreted as 'note_one'.'note_two', where as **notes** are interpreted as 'note_one' or 'note_two'

In [56]:
def get_notes(path):

    notes = []
    durations = []
    for filename in os.listdir(path):
      file = path + "/" + filename
      print("Parsing", file)

      midi = converter.parse(file)
      need_parse = midi.flat.notes

      for i in need_parse:
          if isinstance(i, note.Note):
              notes.append(str(i.pitch))
              durations.append(str(i.duration.quarterLength))
          elif isinstance(i, chord.Chord):
              #This will give chords as numbers: notes.append('.'.join(str(n) for n in i.normalOrder))
              notes.append('.'.join(str(n) for n in i.pitches))
              durations.append(str(i.duration.quarterLength))

    return notes, durations


In [57]:
notes_from_here_please = "/content/drive/My Drive/notebooks/music/training_songs"
notes, durations = get_notes(notes_from_here_please)

Parsing /content/drive/My Drive/notebooks/music/training_songs/breadknife.mid


what does notes look like?

In [71]:
print('printing first part of notes for you...')
notes[1:10]

printing first part of notes for you...


['E-4', 'E4', 'E7.G#6', 'G#3.E4', 'B3', 'B5.G#6', 'A3.E4.C#4', 'B4', 'E-5']

Helper functions

In [59]:
def get_key(val, dictionary):
  """
  Get key from a val in a given dictionary's items
  """
  for key, value in dictionary.items():
        if val == value:
            return key

  return "key DNE"

In [72]:
def list_to_dict(x):
  """
  Converts list to dict or list of lists to dict
  """
  try: #if JUST a list as input
    unique = sorted(set([item for item in x]))
    print(len(unique),' unique notes put into dictionary')
    Dict = {}
    for i in range(0, len(unique)):
      Dict[unique[i]] = i
    #Dict = dict((i, j) for j, i in enumerate(x))
    return Dict

  except: #if LIST input contains LIST (if input is a list of lists...)
    new_x = []
    for i, j in enumerate(x):
      if isinstance(j, list): 
        for k in j:
          new_x.append(k)
      else:
        new_x.append(j)

    return list_to_dict(new_x)

In [73]:
def construct_melody(x, dictionary, sequence_length):
  """
  Constructs "windows" for melody, 
  segments of notes (of len sequence_length) to use as input, 
  since we encode each note using a dictionary. 

  If your collab crashes a lot, this is probably due to a bad dictionary. 
  The autoencoder should be able to handle longer sequences. 
  """
  print('Constructing segments of notes as melodies to use as input, since we encode each note using a dictionary (note_to_int)')
  result = []
  for i in range(0, len(x) - sequence_length):
    sequence_window = x[i:i + sequence_length]
    result.append([dictionary[char] for char in sequence_window])
  return result

Get training data in a format Autoencoder can use

In [62]:
def get_TrainingData(notes, durations, sequence_length):

    print('Desired sequence length is', sequence_length)

    note_to_int = list_to_dict(notes)
    #duration_to_int = list_to_dict(durations)
    print('Note dictionary, \n',note_to_int)
    #print('Duration dictionary, \n',duration_to_int)

    Notes_ = construct_melody(notes, note_to_int, sequence_length)
    #durations_ = construct_melody(durations, duration_to_int, sequence_length)

    Notes_ = np_utils.to_categorical(Notes_).transpose(0,2,1)
    Notes_ = np.array(Notes_, np.float)
    print('notes->Notes_; one-hot encoded; turned into a numpy array')

    return note_to_int, Notes_#, durations_

In [63]:
sequence_length = 60
int_to_note, Notes_ = get_TrainingData(notes, durations, sequence_length)

Desired sequence length is 60
112  unique notes put into dictionary
Note dictionary, 
 {'A1': 0, 'A2': 1, 'A2.C#3': 2, 'A3': 3, 'A3.C#4': 4, 'A3.E-3': 5, 'A3.E4.C#4': 6, 'A3.G#3': 7, 'A4': 8, 'A5': 9, 'A6': 10, 'A6.A5': 11, 'B-2': 12, 'B-4': 13, 'B-5': 14, 'B1': 15, 'B2': 16, 'B2.A3': 17, 'B2.E-3': 18, 'B3': 19, 'B3.C#4.E-4': 20, 'B4': 21, 'B4.B3': 22, 'B4.E4': 23, 'B5': 24, 'B5.B4': 25, 'B5.G#5': 26, 'B5.G#6': 27, 'B6': 28, 'B6.E-6': 29, 'B6.F#6.G#6': 30, 'C#2': 31, 'C#3': 32, 'C#3.F#2': 33, 'C#4': 34, 'C#4.A3': 35, 'C#4.E4': 36, 'C#5': 37, 'C#5.C#4': 38, 'C#5.E4': 39, 'C#5.E5': 40, 'C#6': 41, 'C#6.A5': 42, 'C#6.E5': 43, 'C#6.E6': 44, 'C#7': 45, 'C#7.E6': 46, 'E-3': 47, 'E-3.G#3': 48, 'E-4': 49, 'E-4.F#3': 50, 'E-4.G#3': 51, 'E-5': 52, 'E-5.B4': 53, 'E-6': 54, 'E-6.B5': 55, 'E-6.F#6': 56, 'E-7': 57, 'E-7.E-6': 58, 'E1': 59, 'E2': 60, 'E3': 61, 'E3.E4': 62, 'E4': 63, 'E4.C#4': 64, 'E4.C#5': 65, 'E4.E3': 66, 'E5': 67, 'E6': 68, 'E7.G#6': 69, 'F#1.E4': 70, 'F#2': 71, 'F#3': 72, 'F#3.A3':

Hyperparameters

In [64]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
latent_dim = 2
input_sample = Notes_.shape[0]
input_notes = Notes_.shape[1]
input_dim = input_notes * sequence_length
num_epochs = 10


Run Autoencoder

In [65]:
def get_autoencoder(input_sample, input_notes, input_dim, latent_dim):

    ##Encoder
    EncInput = Input(shape= (input_dim))
    Enc = Dense(latent_dim, activation = 'tanh')(EncInput)
    print(Enc)
    encode = Model(EncInput, Enc)

    ##Decoder
    DecInput = Input(shape= (latent_dim))
    Dec = Dense(input_dim, activation = 'sigmoid')(DecInput)
    print(Dec)
    decode = Model(DecInput, Dec)

    ##Autoencoder
    autoencoder = Model(EncInput, decode(Enc))
    autoencoder.compile(loss = 'binary_crossentropy', optimizer=RMSprop(learning_rate=0.000013))

    return autoencoder, decode

autoencoder, Decode_r = get_autoencoder(input_sample, input_notes, input_dim, latent_dim)
fit_on = Notes_.reshape(input_sample, input_dim)
autoencoder.fit(fit_on, fit_on, epochs=num_epochs)

KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='dense_2/Tanh:0', description="created by layer 'dense_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 6720), dtype=tf.float32, name=None), name='dense_3/Sigmoid:0', description="created by layer 'dense_3'")
Epoch 1/10
13/13 [==============================] - 1s 8ms/step - loss: 0.6931
Epoch 2/10
13/13 [==============================] - 0s 9ms/step - loss: 0.6930
Epoch 3/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6929
Epoch 4/10
13/13 [==============================] - 0s 9ms/step - loss: 0.6928
Epoch 5/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6927
Epoch 6/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6926
Epoch 7/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6925
Epoch 8/10
13/13 [==============================] - 0s 8ms/step - loss: 0.6924
Epoch 9/10
13/13 [==============================] - 0s 8ms/step - loss: 

Decode

In [66]:
def decode(int_to_note, Decode_r):

    ##Computer's melody
    ComputersMelody = Decode_r(np.random.normal(size=(1, latent_dim))).numpy().reshape(input_notes, sequence_length).argmax(0)
    print('Output shape (the length of your song) is \n',np.array(ComputersMelody).shape)
    print('Raw output (prior to passing back to dictionary) is', ComputersMelody)
    MelodyNotes = [get_key(c, int_to_note) for c in ComputersMelody]
    print('After decoding int->note, we get \n',MelodyNotes)
    return MelodyNotes

MelodyNotes = decode(int_to_note, Decode_r)

Output shape (the length of your song) is 
 (60,)
Raw output (prior to passing back to dictionary) is [ 39  79  30  56  17  93  20  76  95  90  75  82  74  79  78 100  28 105
  32  27  88  72  25  28   9   8  23  31 107  29  11  58 100  73  88  34
  90 101  83  60  69  84  55  53   0  47  36  84  24  48  10  80  23  38
   2  90   9   8  93  97]
After decoding int->note, we get 
 ['C#5.E4', 'F#5', 'B6.F#6.G#6', 'E-6.F#6', 'B2.A3', 'G#4', 'B3.C#4.E-4', 'F#4.F#3', 'G#4.B4', 'G#3.E3', 'F#4', 'F#6', 'F#3.E4', 'F#5', 'F#4.G#4', 'G#5.B4', 'B6', 'G#6', 'C#3', 'B5.G#6', 'G#3', 'F#3', 'B5.B4', 'B6', 'A5', 'A4', 'B4.E4', 'C#2', 'G#6.B5.G#5', 'B6.E-6', 'A6.A5', 'E-7.E-6', 'G#5.B4', 'F#3.A3', 'G#3', 'C#4', 'G#3.E3', 'G#5.B4.G#4', 'F3', 'E2', 'E7.G#6', 'F5', 'E-6.B5', 'E-5.B4', 'A1', 'E-3', 'C#4.E4', 'F5', 'B5', 'E-3.G#3', 'A6', 'F#5.E5', 'B4.E4', 'C#5.C#4', 'A2.C#3', 'G#3.E3', 'A5', 'A4', 'G#4', 'G#4.G#5']


Array of notes/chords -> song

Song will save as today's date in your "music" folder

In [89]:
from datetime import datetime

#datetime object containing current date and time
now = datetime.now()
 
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
today = dt_string.replace('/', '').replace(' ', '.').replace(':', '')
print("Current date and time (file will save as this) =", today)	

Current date and time (file will save as this) = 24032021.161432


In [90]:
def give_song(MelodyNotes, today):
    s_obj = stream.Stream()
    s_obj.append(instrument.Piano())

    print(MelodyNotes)

    count = 0
    for x in MelodyNotes:
      if x == "key doesn't exist":
        count += 1
        continue;
      else:
        if '.' in x:
          s_obj.append(chord.Chord(x.replace('.', ' ')))
        else:
          s_obj.append(note.Note(x))

    print('Sorry, I had to erase', count, 'notes or chords that I could not find in your note-to-int dictionary')
    
    #Change to where you want to save this file
    save_to ='/content/drive/My Drive/notebooks/music/'

    s_obj.write('midi', fp=save_to + today + '.mid')
    print('Generated a file in your', save_to, 'path called', str(today)+'.mid')
    return 'Done! Thanks!'

give_song(MelodyNotes, today)

['C#5.E4', 'F#5', 'B6.F#6.G#6', 'E-6.F#6', 'B2.A3', 'G#4', 'B3.C#4.E-4', 'F#4.F#3', 'G#4.B4', 'G#3.E3', 'F#4', 'F#6', 'F#3.E4', 'F#5', 'F#4.G#4', 'G#5.B4', 'B6', 'G#6', 'C#3', 'B5.G#6', 'G#3', 'F#3', 'B5.B4', 'B6', 'A5', 'A4', 'B4.E4', 'C#2', 'G#6.B5.G#5', 'B6.E-6', 'A6.A5', 'E-7.E-6', 'G#5.B4', 'F#3.A3', 'G#3', 'C#4', 'G#3.E3', 'G#5.B4.G#4', 'F3', 'E2', 'E7.G#6', 'F5', 'E-6.B5', 'E-5.B4', 'A1', 'E-3', 'C#4.E4', 'F5', 'B5', 'E-3.G#3', 'A6', 'F#5.E5', 'B4.E4', 'C#5.C#4', 'A2.C#3', 'G#3.E3', 'A5', 'A4', 'G#4', 'G#4.G#5']
Sorry, I had to erase 0 notes or chords that I could not find in your note-to-int dictionary
Generated a file in your /content/drive/My Drive/notebooks/music/ path called 24032021.161432.mid


'Done! Thanks!'